## Ensemble Machine Learning Cookbook

## Stacked Generalization

## Predicted Payment Default

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
url = 'https://raw.githubusercontent.com/YMandCL/Ensemble-Machine-Learning-Cookbook/master/Chapter08/UCI_Credit_Card.csv'
df_creditdata = pd.read_csv(url, index_col = 0)
df_creditdata.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
2,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
3,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
4,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
5,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [3]:
df_creditdata.shape

(30000, 24)

In [4]:
#Assign parameters and target
X = df_creditdata.iloc[:,0:23]
Y = df_creditdata['default.payment.next.month']

#Create train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)

In [5]:
X_train.head(1)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
ID,,,,,,,,,,,,,,,,,,,,,
17819,20000.0,1,3,2,24,0,0,0,0,0,...,19598.0,19701.0,3486.0,199.0,1400.0,2000.0,1000.0,400.0,199.0,0.0


In [6]:
Y_train.head(1)

ID
17819    0
Name: default.payment.next.month, dtype: int64

In [7]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(21600, 23)
(21600,)
(3000, 23)
(3000,)


In [9]:
#Base Learners
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#For Meta-Learner
from sklearn.linear_model import LogisticRegression

In [10]:
#Base Learners
model_1 = GaussianNB()
model_2 = KNeighborsClassifier(n_neighbors=1)
model_3 = DecisionTreeClassifier()

#Train Models
base_learner_1 = model_1.fit(X_train, Y_train)
base_learner_2 = model_2.fit(X_train, Y_train)
base_learner_3 = model_3.fit(X_train, Y_train)

In [11]:
# Use the models to make predictions on validation data
val_prediction_base_learner_1 = base_learner_1.predict(X_val)
val_prediction_base_learner_2 = base_learner_2.predict(X_val)
val_prediction_base_learner_3 = base_learner_3.predict(X_val)

In [12]:
#Use predictions to create new stacked dataset
prediction_test_stack = np.dstack([val_prediction_base_learner_1, val_prediction_base_learner_2, val_prediction_base_learner_3])

In [13]:
#Stack Actual outcomes (Y_Test with prediction stack)
final_train_stack = np.dstack([prediction_test_stack, Y_val])

In [14]:
#Convert final_train_stack array to a DataFrame and Add column names
stacked_train_dataframe = pd.DataFrame(final_train_stack[0,0:5400], columns='NB_VAL KNN_VAL DT_VAL Y_VAL'.split())

print(stacked_train_dataframe.shape)
print(stacked_train_dataframe.head(5))

(5400, 4)
   NB_VAL  KNN_VAL  DT_VAL  Y_VAL
0       1        0       0      0
1       1        0       0      1
2       1        0       0      0
3       1        0       1      1
4       1        0       0      0


<b>Note 5400 observations in 4 columns

In [15]:
# Build the Meta-learner
meta_learner = LogisticRegression()
meta_learner_model = meta_learner.fit(stacked_train_dataframe.iloc[:,0:3],
                                      stacked_train_dataframe['Y_VAL'])

C:\Users\Watson\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [16]:
#Apply base learners on test stack to make predictions
test_prediction_base_learner_1 = base_learner_1.predict(X_test)
test_prediction_base_learner_2 = base_learner_2.predict(X_test)
test_prediction_base_learner_3 = base_learner_3.predict(X_test)

# Create the stacked data
final_test_stack = np.dstack([test_prediction_base_learner_1, test_prediction_base_learner_2,
                              test_prediction_base_learner_3])

In [17]:
#Convert final_test_stack array to Dataframe and add columns
stacked_test_dataframe = pd.DataFrame(final_test_stack[0,0:3000], columns='NB_TEST KNN_TEST DT_TEST'.split())
print(stacked_test_dataframe.shape)
print(stacked_test_dataframe.head(5))

(3000, 3)
   NB_TEST  KNN_TEST  DT_TEST
0        1         0        0
1        1         1        0
2        0         1        1
3        1         1        1
4        1         0        1


<b>Note 3000 observations in 3 columns

In [18]:
#Check Accuracy of Base Learner on original test data
test_prediction_base_learner_1 = base_learner_1.predict(X_test)
test_prediction_base_learner_2 = base_learner_2.predict(X_test)
test_prediction_base_learner_3 = base_learner_3.predict(X_test)

print("Accuracy from GaussianNB:", accuracy_score(Y_test, test_prediction_base_learner_1))
print("Accuracy from KNN:", accuracy_score(Y_test, test_prediction_base_learner_2))
print("Accuracy from Decision Tree:", accuracy_score(Y_test, test_prediction_base_learner_3))

Accuracy from GaussianNB: 0.3913333333333333
Accuracy from KNN: 0.697
Accuracy from Decision Tree: 0.737


In [19]:
#Use Meta-Learner on stacked test data
test_predictions_meta_learner = meta_learner_model.predict(stacked_test_dataframe)
print("Accuracy from Meta Learner:", accuracy_score(Y_test, test_predictions_meta_learner))

Accuracy from Meta Learner: 0.7746666666666666


<b> The Meta Learner accuracy is 77.46%, which is higher than any of the Base Learners.